# NOPT042 Constraint programming: Tutorial 2 – Intro to CP

## Constraint programming aka modeling
Discrete (‘combinatorial’, as opposed to ‘continuous’) optimization, constraint satisfaction
* a form of decision making, many everyday problems
* solve Sudoku
* schedule classes
* schedule trains
* coordinate multi-facility production
* logistics of product transportation
* ...

> Assign values to variables subject to **constraints**, satisfy/optimize.

### We will learn to...
* solve complex problems "without even knowing how"
* state the problem in a high-level language, use a constraint solver to "automagically" solve it (magic explained in the lectures)
* techniques and tricks to build efficient constraint models
* best practices, testing and debugging

### Why constraint programming?
* the ‘holy grail’ of programming: tell the computer what you want, not how to do it
* an order of magnitude easier than programming algorithms
* huge engineering investment in constraint solvers, highly optimized, 
* often faster than your own algorithm would be (especially in “mixed” NP-complete problems), heuristic approach
* easier for molecular biologists to learn to specify their problems in a formal language, than for programmers to learn molecular biology

### History and (folk) etymology
* prográphō ("I set forth as a public notice"), from pró ("towards") + gráphō ("I write")
* program of a political movement
* program of a concert, broadcast programming, tv program
* computer program (1940s)
Independently:
* U.S. Army operational programs (1940s)
* "linear programming" (1946) Maximize $\mathbf{c}^T\mathbf x$ (objective function) subject to $A\mathbf{x}\leq\mathbf{b}$, $\mathbf{x}\geq 0$ (constraints).
* integer programming (1964),
* logic programming (late 1960s)
* constraint logic programming (1987)
* constraint programming (early 1990s)
* Modeling (USA) vs. Modelling (Commonwealth)

### Why Picat?
From a high-level point of view, all constraint programming languages are quite similar. Picat is:
* modern, simple yet powerful
* easier syntax and better utils than SICStus Prolog
* more flexible than MiniZinc
* fast: won several CP competitions (see [Picat homepage](http://picat-lang.org/))


### Recall the basics of constraint programming:

* **decision variables** vs. **parameters**
* **domains**
* **constraints**
* **solution**
* **constraint propagation**
* **search space**
* **decision** vs. **optimization**
* modeling strategies

TODO, see the [slides](http://ktiml.mff.cuni.cz/~bartak/AAAI2017/slides.pdf) (pages 27-34).

In [28]:
%load_ext ipicat

The ipicat extension is already loaded. To reload it, use:
  %reload_ext ipicat


## Structure of a constraint program

Typically, the structure of a constraint program looks like this (code inspired by the [tutorial](http://ktiml.mff.cuni.cz/~bartak/AAAI2017/slides.pdf)):
```
import cp. 

problem(Variables) =>
    declare_variables(Variables),
    post_constraints(Variables).

main =>
    problem(Variables),
    solve(Variables).    
```
Instead of `cp` we can use another solver, e.g. `mip` or `sat`. If the problem is parametrized, then we can pass the `Parameters` as an argument, e.g. `problem(Variables, Parameters)` or we can have a separate function `get_instance(Parameters) = Variables`.


## Introductory examples

### Example: Chinese remainder

After an indecisive battle, general Han Xin wanted to know how
many soldiers of his 42000-strong army remained. In order to
prevent enemy spies hidden among his soldiers to learn the
number, he decided to use modular algebra: He ordered his soldiers
to form rows of 5 and 3 soldiers remained. Then rows of 7; 2
remained. Then rows of 9; 4 remained. Then rows of 11; 10
remained. Finally, rows of 13; 1 remained.

* What are the **parameters** of our problem?
* Identify the decision variables: type, domains (as small as possible).
* What are the constraints? (Are there some implicit constraints?)
* Is this a satisfaction or an optimization problem?
* Find a solution using Picat.
* Is the last condition (rows of 13) needed?


In [50]:
%%picat -n chinese
import cp.

chinese(X) =>
    % variables
    X :: 1..42000,

    % constraints
    X mod 5 #= 3,
    X mod 7 #= 2,
    X mod 9 #= 4,
    
    X mod 11 #= 10.
    
%     X mod 11 #= 10,
%     X mod 13 #= 1.  

In [51]:
%%picat
main =>   
    chinese(X),
    
    % solve
    solve(X),
    
    % output
    println(X).

373



In [52]:
%%picat
main =>   
    chinese(X),
    solve_all(X) = Solutions,
    foreach (Solution in Solutions)
        println(Solution)
    end.

373
3838
7303
10768
14233
17698
21163
24628
28093
31558
35023
38488
41953



In [112]:
!picat chinese-remainder/chinese-remainder.pi
!picat chinese-remainder/chinese-remainder.pi 5 [2,3] [4]
!picat chinese-remainder/chinese-remainder.pi 42000 [5,7,9,11] [3,2,4,10]

35023
*** illegal_arguments

373
3838
7303
10768
14233
17698
21163
24628
28093
31558
35023
38488
41953


In [113]:
!cat chinese-remainder/chinese-remainder.pi

import cp.


chinese(X, Parameters) =>
    % parameters
    [Max, Primes, Moduli] = Parameters,
    
    % here we could test input data
    if Primes.length != Moduli.length then
        throw(illegal_arguments)
    end,

    % variables
    X :: 1..Max,

    % constraints
    foreach(I in 1..Primes.length)        
        X mod Primes[I] #= Moduli[I]
    end.


solve_and_output(Parameters) =>
    chinese(X, Parameters),
    solve_all(X) = Solutions,
    foreach (Solution in Solutions)
        println(Solution)
    end.


main =>
    Parameters = [42000, [5,7,9,11,13], [3,2,4,10,1]],
    solve_and_output(Parameters).


main(Parameters_as_Strings) =>
    Parameters = map(parse_term, Parameters_as_Strings),
    solve_and_output(Parameters).


### Example: Crypt-arithmetic

Solve the crypt-arithmetic puzzle (each letter represents a different base-10 digit):
> SEND + MORE = MONEY

In [114]:
!picat crypt-arithmetic/crypt-arithmetic.pi

*** error(existence_error([c,r,y,p,t,-,a,r,i,t,h,m,e,t,i,c,/,c,r,y,p,t,-,a,r,i,t,h,m,e,t,i,c,.,p,i]),picat)



### Example: Map coloring (only if we have enough time)

Create a model to color the map of Australian states and territories 7 with four colors (cf. The 4-color Theorem). (We exclude the Australian Capital Territory, the Jervis Bay Territory, and the external territories. Map coloring is a special case of [graph coloring](https://en.wikipedia.org/wiki/Graph_coloring), see [this map](map-coloring/countries.jpeg).

![Map of Australia](map-coloring/australia.png)

Let's use the following decision variables:
```
Territories = [WA, NT, SA, Q, NSW, V, T]
```

In [140]:
!picat map-coloring/map-coloring.pi

Western Australia is red.
Northern Territory is green.
South Australia is blue.
Queensland is red.
New South Wales is green.
Victoria is red.
Tasmania is red.


In [141]:
!cat map-coloring/map-coloring.pi

import cp.

color_map(Territories) =>
    % variables
    Territories = [WA, NT, SA, Q, NSW, V, T],
    Territories :: 1..4,

    % constraints
    WA #!= NT,
    WA #!= SA,
    NT #!= SA,
    NT #!= Q,
    SA #!= Q,
    SA #!= NSW,
    SA #!= V,
    Q  #!= NSW,
    V  #!= NSW.

% optional: symmetry breaking
precolor(Territories) =>
    WA #= 1,
    NT #= 2,
    SA #= 3.

% optional: better output than `println(Territories)` (we could also use a map, i.e. a dictionary)
output(Territories) =>
    Color_names = ["red", "green", "blue", "yellow"],
    Territory_names = ["Western Australia", "Northern Territory", "South Australia", "Queensland", "New South Wales", "Victoria", "Tasmania"],
    foreach(I in 1..Territories.length)
        writef("%s is %s.\n", Territory_names[I], Color_names[Territories[I]])
    end.


main =>
    color_map(Territories),
    precolor(Territories), % optional
    solve(Territories),
    
    % println(Territories)    
    output(Territories).


What is wrong with this model? We always want to separate the model from the data. (See the exercises below.)

## Exercises

### Exercise: Pythagorean triples
1. Generate all Pythagorean triples up to a given parameter, i.e. positive integers such that $a^2+b^2=c^2$, where $a\leq b$ (an example of [symmetry breaking](https://en.wikipedia.org/wiki/Symmetry-breaking_constraints)). 
2. Modify your program to accept the flag -c to output the number of solutions.
```
picat pythagorean.pi 42
picat pythagorean.pi -c 42
```

### Exercise: Crypt-aritmetic with carry bits
Write a better constraint model for the SEND+MORE=MONEY crypt-arithmetic puzzle based on carry bits. Why is it better?

### Exercise: Graph-coloring (only if we have enough time)
1. Write a program that solves the (directed) graph 3-coloring problem with a given number of colors and a given graph. The graph is given by a list of edges, each edge is a 2-element list. We assume that vertices of the graph are $1,\dots,n$ where $n$ is the maximum number appearing in the list. 
2. Generalize your program to graph $k$-coloring where $k$ is a positive integer given on the input.
3. Modify your program to accept the incidence matrix (a 2D array) instead of the list of edges.
4. Add the flag `-n` to output the minimum number of colors (the chromatic number) of a given graph.
For example:
```
picat graph-coloring.pi [[1,2],[2,3],[3,4],[4,1]]
picat graph-coloring.pi [[1,2],[2,3],[3,1]] 4
picat graph-coloring.pi "{{0,1,1},{1,0,1},{1,1,0}}" 4
picat graph-coloring.pi -n [[1,2],[2,3],[3,4],[4,1]]
```

### Homework: general crypt-arithmetic

1. Write a program that accepts a crypt-arithmetic puzzle on the input and outputs a solution as a string where letters are replaced by digits, e.g.
```
picat crypt-arithmetic.pi SEND MORE MONEY
```
should output `9567 + 1085 = 10652` (since this is the only solution). Don't forget to include the spaces.

2. Ignore case, e.g. accept also:
```
picat crypt-arithmetic.pi Donald Gerard Robert
picat crypt-arithmetic.pi baijjajiiahfcfebbjea dhfgabcdidbiffagfeje gjegacddhfafjbfiheef
```
(source of the last instance: [Hakan Kjellerstrand's library](http://www.hakank.org/))

2. Modify your program to accept the flag `-c` to output the number of solutions instead, e.g.
```
picat crypt-arithmetic.pi -c send more money
```
should output `1`.

Your model must be reasonably efficient, e.g. use the carry bit implementation.

See the assignment on GitHub Classroom.

## Playground